In [73]:
import numpy as np
from tqdm import tqdm
from itertools import product
filename = "input_21"
#filename = "example_21.txt"
with open(filename) as f:
    input = f.read().split("\n")

## Part 1:

In [74]:
def input_to_keypad(input):
    numeric_pad = np.asarray([[7,8,9],[4,5,6],[1,2,3],["no",0,"A"]])
    i0, j0 = np.where(numeric_pad == "A")
    sequences = [""]
    for char in input:
        i1, j1 = np.where(numeric_pad == char)
        di, dj = (i1 - i0).item(), (j1 - j0).item()
        to_add = []
        # l = left, r = right, u = up, d = down, p = press
        if di < 0: to_add.append("u" * -di)
        if dj > 0: to_add.append("r" * dj)
        if di > 0: to_add.append("d" * di)
        if dj < 0: to_add.append("l" * -dj)
        condition = (i0 + di == 3 and j0 == 0) or (j0 + dj == 0 and i0 == 3)
        if len(to_add) > 1 and not condition:
            perms = ("".join(to_add), "".join(reversed(to_add)))
        else:
            perms = ("".join(to_add),)
        for sequence in sequences.copy():
            sequences += [sequence + perm + "A" for perm in perms]
            sequences.remove(sequence)
        i0, j0 = i1, j1
    return sequences

In [75]:
def keypad_to_keypad(keypad):
    dirmap = {"no":np.array((0,0)), 
          "u":np.array((0,1)), 
          "A":np.array((0,2)),
          "l":np.array((1,0)),
          "d":np.array((1,1)),
          "r":np.array((1,2))}
    current = "A"
    sequence = []
    for char in keypad:
        dy, dx = dirmap[char] - dirmap[current]
        # order: down before left, right before up
        if dy > 0: sequence.extend(["d"] * dy)
        if dx < 0: sequence.extend(["l"] * -dx)
        if dx > 0: sequence.extend(["r"] * dx)
        if dy < 0: sequence.extend(["u"] * -dy)
        sequence.append("A")
        current = char
    return tuple(sequence)

In [ ]:
c = 0
for code in input:
    keypads = input_to_keypad(code)
    sequences = [keypad_to_keypad(keypad_to_keypad(keypad)) for keypad in keypads]
    l = min([len(sequence) for sequence in sequences])
    print(l)
    c += l * int(code[:-1])
c

## Part 2:

In [ ]:
""""
c = 0
for code in input:
    keypads = [list(pad) for pad in input_to_keypad(code)]
    sequences = []
    for keypad in keypads:
        sequence = ["A"] + keypad
        for _ in tqdm(range(26)):
            new_sequence = ["A"]
            for i,j in zip(sequence[:-1], sequence[1:]):
                new_sequence.extend(map[i,j])
            sequence = new_sequence.copy()
            print(len(sequence))
        sequences.append(sequence)
    l = min([len(sequence) for sequence in sequences])
    print(l)
    c += l * int(code[:-1])
c
"""

In [78]:
map = {}
for char1 in "Aurld":
    for char2 in "Aulrd":
        path = keypad_to_keypad(char1 + char2)
        idx = path.index("A")
        map[(char1,char2)] = path[idx+1:]

In [79]:
# Need to swap d <--> l when going from A to d
# and r <--> u when going from d to A
map[("A","d")] = ("l","d","A")
map[("d","A")] = ("u","r","A")

In [80]:
def solution_2(codes, map, pad_amount):
    c = 0
    for code in codes:
        # convert numeric input code into (possible) directional keypad codes
        keypads = [list(pad) for pad in input_to_keypad(code)]

        lengths = []
        for keypad in keypads:
            # start from A, count multiplicities of character pairs
            sequence = ["A"] + keypad
            multiplicities = dict([(comb, 0) for comb in map.keys()])
            for pair in zip(sequence[:-1], sequence[1:]):
                multiplicities[pair] += 1

            # evolve every character pair, count multiplicites
            for _ in range(pad_amount):
                new_multiplicities = dict([(comb, 0) for comb in map.keys()])
                for pair in multiplicities.keys():
                    new_sequence = ("A",) + map[pair]
                    new_pairs = list(zip(new_sequence[:-1], new_sequence[1:]))
                    for new_pair in new_pairs:
                        new_multiplicities[new_pair] += multiplicities[pair]
                multiplicities = new_multiplicities.copy()
            lengths.append(sum(multiplicities.values()))

        l = min(lengths)
        c += l * int(code[:-1])
    print(c)

In [ ]:
solution_2(input, map, 25)